In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print(tf.__version__)


2.8.0


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
(train_images, val_images, train_labels, val_labels) = train_test_split(train_images, train_labels, test_size=0.33, random_state=42)


4431872/4422102 [==============================] - 0s 0us/step


In [3]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [9]:
# Preprocess both train and test
train_images = train_images / 255.0
val_images = val_images /255.0
test_images = test_images / 255.0


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

2022-04-23 15:45:08.959606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 15:45:08.963906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 15:45:08.964246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 15:45:08.965161: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 100,609
Trainable params: 100,609
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy']
)



In [24]:
import os
checkpoint_path = "results/cnn-training-220423/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq=3*32,
                                                 verbose=1
)


In [25]:

model.fit(
    train_images, 
    train_labels, 
    batch_size = 32,
    validation_data=(val_images, val_labels), 
    epochs=10,
    callbacks=[cp_callback]
)


Epoch 1/10
  89/1257 [=>............................] - ETA: 2s - loss: 0.0584 - accuracy: 0.9726
Epoch 1: saving model to results/cnn-training-220423/cp-0001.ckpt
 180/1257 [===>..........................] - ETA: 1s - loss: 0.0603 - accuracy: 0.9727
Epoch 1: saving model to results/cnn-training-220423/cp-0001.ckpt
 268/1257 [=====>........................] - ETA: 1s - loss: 0.0606 - accuracy: 0.9735
Epoch 1: saving model to results/cnn-training-220423/cp-0001.ckpt
 353/1257 [=======>......................] - ETA: 1s - loss: 0.0596 - accuracy: 0.9738
Epoch 1: saving model to results/cnn-training-220423/cp-0001.ckpt
 476/1257 [==========>...................] - ETA: 1s - loss: 0.0588 - accuracy: 0.9751
Epoch 1: saving model to results/cnn-training-220423/cp-0001.ckpt
 562/1257 [============>.................] - ETA: 1s - loss: 0.0596 - accuracy: 0.9750
Epoch 1: saving model to results/cnn-training-220423/cp-0001.ckpt
 650/1257 [==============>...............] - ETA: 1s - loss: 0.0592 - a

KeyboardInterrupt: 

In [83]:
pred = model.predict(train_images)


In [79]:
B = np.where(pred>0.5, 1, 0).reshape((-1)).astype(np.float32)

In [84]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce(train_labels, pred)

In [45]:


np.sum(A*B)

53140

In [33]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)


313/313 - 1s - loss: 0.3527 - accuracy: 0.8796 - 782ms/epoch - 2ms/step

Test accuracy: 0.8795999884605408


In [34]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])


In [35]:
predictions = probability_model.predict(test_images)


In [41]:
for pair in zip(list(predictions[1]), class_names):
    print(pair)


(5.9892518e-05, 'T-shirt/top')
(3.6508465e-14, 'Trouser')
(0.99979645, 'Pullover')
(5.390233e-10, 'Dress')
(8.579558e-05, 'Coat')
(8.918108e-12, 'Sandal')
(5.7978446e-05, 'Shirt')
(2.033656e-12, 'Sneaker')
(2.9201763e-12, 'Bag')
(5.991143e-14, 'Ankle boot')


In [44]:
# Add the image to a batch where it's the only member.
img = test_images[1]

img = (np.expand_dims(img,0))

predictions_single = probability_model.predict(img)


In [45]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)
plt.show()


NameError: name 'plot_value_array' is not defined